<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Предообработка-данных" data-toc-modified-id="Предообработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предообработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# импортирую библиотеки для решения поставленных задач
import pandas as pd
import numpy as np
from sklearn.datasets import make_spd_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# открываем файл
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
# посмотрим  первые 5 верхних строк датасета
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# посмотрим  первые 5 нижних строк датасета
df.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [5]:
# посмотрим информацию в целом о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


**Вывод:**
- наименования атрибутов представлены на русском языке, предлагается переименовать на английский;
- не соблюден змеиный регистр (нижний регистр);
- значений NaN (отсутствующее в ячейке число), None (нечисловой тип 'NoneType') при первичном осмотре не установлены;
- типы данных соответствуют логике атрибутов;
- необходимо проверить данные на наличие дубликатов.

### Предообработка данных

In [6]:
# изменим на английский язык названия столбцов
# сохраним измененный датасет в переменную df_eng
df_eng = df.set_axis(['gender','age','salary','family_member','insurance_payments'],axis = 'columns',inplace = False)

In [7]:
# проведем подсчёт пропущенных значений
df_eng.isna().sum()

gender                0
age                   0
salary                0
family_member         0
insurance_payments    0
dtype: int64

In [8]:
# проведем подсчет суммы дубликатов если они конечно есть
df_eng.duplicated().sum()

153

*153 дубликата выявлено.Оносительно 5 тысяч строк в датасете количество выявленных дубликатов незначительно и их удаление не скажется на качестве данных для проведения анализа. Удаляем дубликаты.*

In [9]:
# удаление дубликатов
df_personal_data = df_eng.drop_duplicates().reset_index(drop=True)

In [10]:
# проведем подсчет суммы дубликатов после их удаления
df_personal_data.duplicated().sum()

0

In [11]:
# посмотрим информацию в целом о пройденном предобработку датасете
df_personal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              4847 non-null   int64  
 1   age                 4847 non-null   float64
 2   salary              4847 non-null   float64
 3   family_member       4847 non-null   int64  
 4   insurance_payments  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


**Вывод:** 

*Предообработка проведена, намеченные планы по визуальному осмотру данных, в том числе по итогам осмотра с помощью специнструментов проделана. Можем приступать к следующему этапу работы.*

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос** 

*Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)*
- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** b. Не изменится.

**Обоснование:** 
$$
a = Xw = a' = XPw' ((XP)^TXP)^{-1}(XP)^Ty=(P^TX^TXP)^{-1}=P^{-1}(X^TX)^{-1}(P^T)^{-1}(P^T)(X^Ty)=P^{-1}(X^TX)^{-1}(X^Ty)=(P^{-1}w)
$$

*параметры линейной регрессии в исходной задаче и в преобразованной связаны через веса матрицы, которая может быть умножена на обратную матрицу*

## Алгоритм преобразования

**Алгоритм:**

- создаем случайную обратимую матрицу Y
- умножаем признаки матрицы Х на матрицу Y

**Обоснование**

$$ XYY^{-1}=X$$

## Проверка алгоритма

In [12]:
# подготовим фичи
features = df_personal_data.drop('insurance_payments', axis=1)

In [13]:
# подготовим целевые признаки
target = df_personal_data['insurance_payments']

In [14]:
# используем библиотеку from sklearn.datasets import make_spd_matrix и создадим слуайную, обратимую матрицу
matrix = make_spd_matrix(n_dim=4, random_state=12345)

In [15]:
# умножаем признаки матрицы Х (features) на матрицу Y (matrix) 
matrix_x = (features.dot(matrix)) 

In [16]:
# попробуем восстановить данные и обратить процесс назад
(matrix_x @ np.linalg.inv(matrix)).head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,5.955746e-12,46.0,38000.0,1.000000e+00
2,-9.965056e-13,29.0,21000.0,-1.430738e-12
3,-1.023961e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-2.136585e-12


In [17]:
# для сравнения посмотрим первые пять строк features
features.head()

,gender,age,salary,family_member
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


**Вывод:**

*Данные в таблцие практически идентичны*

**Следующая задача (согласно Инструкции): запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.**

In [18]:
# чтоб сократить количество кода создам функцию по разбиению датасета на обучающую и тестовую выборки
# по фичам и таргетам
def t_t_splitting(features, target):
    features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                     test_size=0.25, random_state=12345)
    return features_train, features_test, target_train, target_test

In [19]:
# применим функцию к созданным фичам и таргетам
features_train, features_test, target_train, target_test = t_t_splitting(features, target)

In [20]:
features_matrix_x_train, features_matrix_x_test, target_matrix_x_train, target_matrix_x_test = t_t_splitting(matrix_x, target)

**Проведем сравнение качества моделей до и после преобразования. Применим метрику R2.**

In [21]:
# до преобразования
model = LinearRegression()
model.fit(features_train, target_train)
prediction = model.predict(features_test)
print(r2_score(target_test, prediction))

0.42307727492147296


In [22]:
# после преобразования
model = LinearRegression()
model.fit(features_matrix_x_train, target_matrix_x_train)
prediction_x = model.predict(features_matrix_x_test)
print(r2_score(target_matrix_x_test, prediction_x))

0.42307727492177594


**Общий вывод:**
- по результатам первичного осмотра датасета проведена предообработка данных для того, чтобы более качественно провести исследование;
- далее создана случайная обратимая матрица. 
- умножили признаки на нашу обратимую матрицу. Таким образом мы зашифровали наши признаки.
-  далее обучили модель и посмотрели изменится ли r2, если обучить модель на зашифрованных признаках. 
- проведено сравнение качества моделей до и после преобразования в результате которого установлено, что до и после преобразования значение r2 не изменились.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования